In [2]:
import kagglehub

path = kagglehub.dataset_download("mateibejan/15000-gutenberg-books")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Antisha\.cache\kagglehub\datasets\mateibejan\15000-gutenberg-books\versions\5


In [ ]:
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from gutenbergpy.gutenbergcache import GutenbergCache
from gutenbergpy.textget import get_text_by_id

PATH_TO_METADATA = r"C://Users//Antisha//.cache//kagglehub//datasets//mateibejan//15000-gutenberg-books//versions//5//gutenberg_metadata.csv"

def clean_text(text):
    cleaned_listed_text = []
    listed_text = list(text)

    for iter in range(len(listed_text) - 1):
        if (listed_text[iter] == '\\' and listed_text[iter + 1] == 'n') or \
            (listed_text[iter] == 'n' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\' and listed_text[iter + 1] == 'r' or \
            (listed_text[iter] == 'r' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\' and listed_text[iter + 1] == 't' or \
            (listed_text[iter] == 't' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\':
            continue
        else:
            cleaned_listed_text.append(listed_text[iter])

    cleaned_text = ''.join([str(char) for char in cleaned_listed_text])
    return cleaned_text

df_metadata = pd.read_csv(PATH_TO_METADATA)

data = {'Author': None, 'Title': None, 'Link': None, 'ID': None, 'Bookshelf': None, 'Text': None}


for key, row in df_metadata.iterrows():
    if data['Author'] == None:
        data['Author'] = [row['Author']]
    else:
        data['Author'].append(row['Author'])
    
    if data['Title'] == None:
        data['Title'] = [row['Title']]
    else:
        data['Title'].append(row['Title'])
    
    if data['Link'] == None:
        data['Link'] = [row['Link']]
    else:
        data['Link'].append(row['Link'])
    
    book_id = int(row['Link'].split('/')[-1])

    if data['ID'] == None:
        data['ID'] = [book_id]
    else:
        data['ID'].append(book_id)
    
    if data['Bookshelf'] == None:
        data['Bookshelf'] = [row['Bookshelf']]
    else:
        data['Bookshelf'].append(row['Bookshelf'])

    try:
        text = get_text_by_id(book_id)
        text = clean_text(str(text))
    except:
        try: 
            page = requests.get(row['Link'])
            soup = BeautifulSoup(page.content, 'html.parser')
            text_link = 'http://www.gutenberg.org' + soup.find_all("a", string="Plain Text UTF-8")[0]['href']
            http_response_object = urlopen(text_link)

            text = http_response_object.read().decode('utf-8')
            text = clean_text(str(text))
        except:
            print("Couldn't acquire text for " + row['Title'] + ' with ID ' + str(book_id) + '. Link: ' + row['Link'])
            
    if data['Text'] == None:
        data['Text'] = [' '.join(text.split(' '))]
    else:
        try:
            data['Text'].append(' '.join(text.split(' ')))
        except:
            data['Text'].append(None)
            print("Couldn't save data for " + row['Title'] + ' with ID ' + str(book_id) + '. Link: ' + row['Link'])

df_data = pd.DataFrame(data, columns = ['Title', 'Author', 'Link', 'ID', 'Bookshelf', 'Text'])
df_data.to_csv(os.path.join(os.getcwd(), 'data', 'gutenberg_data.csv'), index=False)

Couldn't acquire text for Ragged Dick with ID 20689. Link: http://www.gutenberg.org/ebooks/20689
Couldn't acquire text for The Owl and the Pussycat with ID 23897. Link: http://www.gutenberg.org/ebooks/23897
Couldn't acquire text for Don Quixote, Volume 1 with ID 28842. Link: http://www.gutenberg.org/ebooks/28842
Couldn't acquire text for The Gift of the Magi with ID 22440. Link: http://www.gutenberg.org/ebooks/22440
Couldn't acquire text for The Garden Party, and Other Stories with ID 26463. Link: http://www.gutenberg.org/ebooks/26463


In [ ]:
df_data